In [1]:
# 필요한 라이브러리를 불러온다
import requests
import openai
from PIL import Image

import base64
import openai
import pillow_heif
import pandas as pd
import os
import torch

In [2]:
# OpenAI 버전 확인
print(openai.__version__)

0.28.0


In [ ]:
# OpenAI API 키 설정
openai.api_key = '.'

### 사용자가 찍은 사진을 업로드

In [4]:
df = pd.read_csv("C:/Users/KimDongyoung/Desktop/food_names.csv")
# 맨 앞 행 4개를 제거
df = df.iloc[4:].reset_index(drop=True)
food_names = df['food_name'].tolist()
food_names

['비빔국수', '닭갈비', '오코노미야키']

### 연습

In [ ]:
# def start_conversation(food_names):
#     # 음식 목록을 포함한 사용자에게 대화 시작
#     food_message = (
#         f"다음은 최근에 맛있게 먹었던 음식 사진이야: {', '.join(food_names)}. 추가적으로 사용자의 음식 취향을 이해하기 위해 몇 가지 질문을 할 거야."
#     )

#     # 카테고리별 질문 설정
#     categories = [
#         "주 식재료는 무엇인가요? (예: 닭고기, 고등어, 새우 등)",
#         "못 먹는 재료가 있나요? (예: 달걀, 우유, 밀가루 등)",
#         "매운, 안매운 중 선택",
#         "따뜻한, 시원한 중 선택",
#         "아침, 점심, 저녁, 야식 중 언제 먹을 건가요?",
#         "요리시간 20분 이내, 1시간 이내 선택",
#     ]

#     # 대화 메시지 준비
#     messages = [
#         {"role": "system", "content": "너는 사용자의 음식 취향을 이해하는 요리 전문가야."},
#         {"role": "user", "content": food_message}
#     ]

#     # 카테고리별로 질문을 던지고 사용자 응답 받기
#     user_responses = {}  # 사용자 응답 저장
#     for category in categories:
#         category_message = f"{category}을(를) 입력해 주세요:"
#         print(category_message)  # 사용자에게 질문 출력

#         user_input = input(f"{category}: ")  # 사용자로부터 입력 받기
#         user_responses[category] = user_input  # 사용자의 응답 저장

#     # 사용자 응답을 조건문을 활용하여 하나의 문장으로 결합
#     combined_responses = (
#         f"주 식재료는 {user_responses[categories[0]]}을 사용하고 "
#         f"못 먹는 식재료는 {user_responses[categories[1]]}입니다. "
#         f"{user_responses[categories[2]]} 음식을 먹고 싶으며 "
#         f"{user_responses[categories[3]]} 음식을 먹고 싶어 합니다. "
#         f"{user_responses[categories[4]]}에 먹을 음식이며 "
#         f"요리시간은 {user_responses[categories[5]]}입니다."
#     )
    
#     print(combined_responses)

#         # 대화 내용에 사용자의 입력 추가
#     user_message = f"The user selected: {combined_responses}"
#     messages.append({"role": "user", "content": user_message})

#     # 사용자 응답을 바탕으로 음식 추천 요청
#     recommendation_message = "({combined_responses}) 사용자의 응답을 바탕으로 5개의 음식을 추천해주세요."
#     messages.append({"role": "user", "content": recommendation_message})

#     # 음식 추천 받기
#     response = openai.ChatCompletion.create(
#         model="gpt-4o-mini",
#         messages=messages
#     )

#     # 추천된 음식 목록 출력
#     recommended_meals = response.choices[0].message['content']
#     print("Recommended Meals:")
#     print(recommended_meals)
    
#     # 사용자가 선택한 음식 입력 받기
#     selected_meal = input("이 중 요리하고 싶은 음식을 입력해주세요: ")

#     # 선택한 음식의 레시피 요청
#     recipe_message = f"{selected_meal} 이 음식의 요리재료와 레시피를 알려주는데 이 때 요리를 못하는 사람도 이해할 수 있게 자세히 작성해줘."
#     messages.append({"role": "user", "content": recipe_message})

#     # 레시피 받기
#     recipe_response = openai.ChatCompletion.create(
#         model="gpt-4o-mini",
#         messages=messages
#     )

#     # 최종 응답 내용 반환
#     return recipe_response.choices[0].message['content']


In [ ]:
# recipe_content = start_conversation(food_names)
# print("선택한 음식의 레시피 공개!:")
# print(recipe_content)

### LLM을 활용하여 사용자가 찍은 음식 사진과 지금 현재 먹고 싶은 음식 정보를 입력하면 음식을 추천해주고 레시피도 생성해주는 모델

In [ ]:
def start_conversation(food_names):
    # 음식 목록을 포함한 사용자에게 대화 시작
    food_message = (
        f"다음은 최근에 맛있게 먹었던 음식들이야: {', '.join(food_names)}. 추가적으로 사용자가 요리 해먹고 싶은 음식을 추천하기 위해 몇 가지 질문을 할 거야."
    )

    # 카테고리별 질문 설정
    categories = [
        "주 식재료는 무엇인가요? (예: 닭고기, 고등어, 새우 등)",
        "못 먹는 재료가 있나요? (예: 달걀, 우유, 밀가루 등)",
        "매운, 안매운 중 선택",
        "따뜻한, 시원한 중 선택",
        "아침, 점심, 저녁, 야식 중 언제 먹을 건가요?",
        "요리시간 20분 이내, 1시간 이내 선택",
    ]

    # 대화 메시지 준비
    messages = [
        {"role": "system", "content": "너는 요리 전문가로 사용자가 최근에 먹은 음식들을 바탕으로 비슷한 음식을 추천해줄거야."},
        {"role": "user", "content": food_message}
    ]

    # 카테고리별로 질문을 던지고 사용자 응답 받기
    user_responses = {}  # 사용자 응답 저장
    for category in categories:
        category_message = f"{category}을(를) 입력해 주세요:"
        print(category_message)  # 사용자에게 질문 출력

        user_input = input(f"{category}: ")  # 사용자로부터 입력 받기
        user_responses[category] = user_input  # 사용자의 응답 저장

    # 사용자 응답을 조건문을 활용하여 하나의 문장으로 결합
    combined_responses = (
        f"주 식재료는 {user_responses[categories[0]]}을 사용하고 "
        f"못 먹는 식재료는 {user_responses[categories[1]]}입니다. "
        f"{user_responses[categories[2]]} 음식을 먹고 싶으며 "
        f"{user_responses[categories[3]]} 음식을 먹고 싶어 합니다. "
        f"{user_responses[categories[4]]}에 먹을 음식이며 "
        f"요리시간은 {user_responses[categories[5]]}입니다."
    )
    
    print(combined_responses)

    # 대화 내용에 사용자의 입력 추가
    user_message = f"The user provided the following preferences: {combined_responses}"
    messages.append({"role": "user", "content": user_message})

    # 사용자 응답을 바탕으로 음식 추천 요청
    recommendation_message = f"({combined_responses}) 사용자의 응답을 바탕으로 3개의 음식을 추천해주세요.추천할 때 음식 이름만 적어주세요."
    messages.append({"role": "user", "content": recommendation_message})

    # 음식 추천 받기
    response1 = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # 추천된 음식 목록 출력
    recommended_meals = response1.choices[0].message['content']
    print("Recommended Meals based on user preferences:")
    print(recommended_meals)

    # food_names를 바탕으로 2개의 음식 추천 요청
    food_names_message = f"Based on the following food names: {', '.join(food_names)}, please recommend 2 meals that are not in the following list: {', '.join(recommended_meals)}."
    messages.append({"role": "user", "content": food_names_message})

    # 음식 추천 받기
    response2 = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # 추천된 음식 목록 출력
    additional_meals = response2.choices[0].message['content']
    print("첨부한 음식 사진을 바탕으로 추천된 음식:")
    print(additional_meals)

    # 사용자가 선택한 음식 입력 받기
    selected_meal = input("이 중 요리하고 싶은 음식을 입력해주세요: ")

    # 선택한 음식의 레시피 요청
    recipe_message = user_message = (
        f"{selected_meal} 이 음식의 요리재료와 레시피를 알려주는데 이 때 요리를 못하는 사람도 이해할 수 있게 자세히 작성해줘. "
        "예시: "
        "주재료\n"
        "닭고기: 500g (닭다리살 또는 가슴살 추천)\n"
        "양배추: 1/4개 (먹기 좋은 크기로 자름)\n"
        "고구마: 1개 (껍질을 벗기고 얇게 썰기)\n"
        "당근: 1개 (얇게 채 썰기)\n"
        "대파: 1대 (어슷하게 썰기)\n"
        "버섯: 100g (표고버섯, 느타리버섯 등, 먹기 좋은 크기로 자름)\n"
        "양념 재료\n"
        "고추장: 2큰술\n"
        "고춧가루: 1큰술 (매운 정도에 따라 조절)\n"
        "간장: 2큰술\n"
        "다진 마늘: 1큰술\n"
        "설탕: 1큰술 (단맛을 원하면 추가)\n"
        "참기름: 1큰술\n"
        "후춧가루: 약간\n"
        "요리방법\n"
        "1단계: 재료 손질\n"
        "닭고기: 닭고기를 한 입 크기로 자른 후, 뼈가 있는 부위를 사용할 경우 뼈를 제거하고 살만 남깁니다.\n"
        "채소: 양배추는 굵게 썰고, 고구마는 얇게 썰어 익히기 쉽게 준비한다. 당근은 채 썰고, 대파는 어슷하게 썬다. 버섯은 먹기 좋은 크기로 자릅니다.\n"
        "2단계: 양념장 만들기\n"
        "큰 볼에 고추장, 고춧가루, 간장, 다진 마늘, 설탕, 참기름, 후춧가루를 넣고 잘 섞어 양념장을 만든다. 이때, 양념의 맛을 보고 필요에 따라 간장이나 설탕을 추가해 조절할 수 있습니다.\n"
        "3단계: 닭고기 재우기\n"
        "손질한 닭고기를 양념장에 넣고 잘 버무린 후, 랩으로 덮거나 밀폐 용기에 담아 최소 30분 이상 재워둔다. 가능하다면 냉장고에서 1시간 이상 재우면 더욱 맛이 깊어집니다.\n"
        "4단계: 볶기\n"
        "팬 준비: 큰 팬이나 철판에 식용유를 두르고 중불로 달굽니다.\n"
        "닭고기 볶기: 재워둔 닭고기를 팬에 넣고 볶기 시작한다. 닭고기가 겉면이 익을 때까지 볶습다. 이때, 닭고기를 자주 뒤집어 주어 고르게 익도록 합니다.\n"
        "채소 추가: 닭고기가 어느 정도 익으면 양배추, 고구마, 당근, 대파, 버섯을 차례로 넣고 함께 볶습니다. 채소가 숨이 죽고 고구마가 익을 때까지 볶습니다. 필요에 따라 물을 조금 추가해도 좋습니다.\n"
        "5단계: 완성\n"
        "모든 재료가 잘 익고 양념이 고루 배어들면 불을 끕니다. 이때, 맛을 보고 필요에 따라 소금이나 후춧가루로 간을 추가할 수 있습니다.\n"
        "접시에 담고, 원한다면 쌈채소(상추, 깻잎 등)와 함께 제공합니다. 추가로, 쌈장이나 마늘, 고추를 곁들여 먹으면 더욱 맛있습니다.\n"
        "팁\n"
        "매운 맛 조절: 고춧가루의 양을 조절하거나 청양고추를 추가하여 매운 맛을 조절할 수 있습니다.\n"
        "추가 재료: 떡, 면, 또는 다른 채소(브로콜리, 피망 등)를 추가하여 다양하게 즐길 수 있습니다.\n"
        "조리 도구: 철판에서 조리하면 더욱 맛있고, 테이블에서 직접 볶아 먹는 재미가 있습니다."
    )
    messages.append({"role": "user", "content": recipe_message})

    # 레시피 받기
    recipe_response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # 최종 응답 내용 반환
    return recipe_response.choices[0].message['content']

# 사용 예시
df = pd.read_csv("C:/Users/KimDongyoung/Desktop/food_names.csv")  # 사용자가 찍은 음식 사진
# 맨 앞 행 4개를 제거
df = df.iloc[4:].reset_index(drop=True)
food_names = df['food_name'].tolist()     # 음식 이름 리스트로 변환
food_names

recipe_content = start_conversation(food_names) 
print("선택한 음식의 레시피 공개!:")
print(recipe_content)

주 식재료는 무엇인가요? (예: 닭고기, 고등어, 새우 등)을(를) 입력해 주세요:
못 먹는 재료가 있나요? (예: 달걀, 우유, 밀가루 등)을(를) 입력해 주세요:
매운, 안매운 중 선택을(를) 입력해 주세요:
따뜻한, 시원한 중 선택을(를) 입력해 주세요:
아침, 점심, 저녁, 야식 중 언제 먹을 건가요?을(를) 입력해 주세요:
요리시간 20분 이내, 1시간 이내 선택을(를) 입력해 주세요:
주 식재료는 닭고기을 사용하고 못 먹는 식재료는 없음입니다. 매운 음식을 먹고 싶으며 따뜻한 음식을 먹고 싶어 합니다. 저녁에 먹을 음식이며 요리시간은 20분 이내입니다.
Recommended Meals based on user preferences:
1. 매운 닭고기 볶음
2. 닭고기 김치찌개
3. 닭갈비 볶음밥
첨부한 음식 사진을 바탕으로 추천된 음식:
1. 매운 닭고기 스튜  
2. 닭고기 고추장 볶음  
선택한 음식의 레시피 공개!:
**매운 닭고기 스튜**

**주재료**
- 닭고기: 500g (닭다리살 또는 가슴살 추천)
- 감자: 1개 (껍질을 벗기고 큼직하게 썰기)
- 양파: 1개 (굵게 다지기)
- 파프리카: 1개 (어슷하게 썰기)
- 당근: 1개 (먹기 좋은 크기로 썰기)
- 대파: 1대 (어슷하게 썰기)

**양념 재료**
- 고추장: 2큰술
- 고춧가루: 1큰술 (매운 정도에 따라 조절)
- 간장: 2큰술
- 다진 마늘: 1큰술
- 설탕: 1큰술 (단맛을 원하면 추가)
- 물: 1컵
- 후춧가루: 약간
- 식용유: 1큰술

**요리방법**

1. **재료 손질**
   - **닭고기**: 닭고기를 한 입 크기로 자르고, 여분의 지방이나 뼈를 제거합니다.
   - **채소**: 감자는 껍질을 벗기고 큼직하게 썰고, 양파는 곱게 다지고, 파프리카와 당근은 먹기 좋은 크기로 썰어줍니다. 대파는 어슷하게 썰어 준비합니다.

2. **양념장 만들기**
   - 큰 볼에 고추장, 고춧가루, 간장, 다진 마늘, 설탕, 후춧가루를 넣고 잘 섞어 양념장을 만듭니다